<a href="https://colab.research.google.com/github/jsedoc/ConceptorDebias/blob/master/EEC_with_CN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://www.saifmohammad.com/WebDocs/EEC/Equity-Evaluation-Corpus.zip

--2019-02-17 19:25:14--  http://www.saifmohammad.com/WebDocs/EEC/Equity-Evaluation-Corpus.zip
Resolving www.saifmohammad.com (www.saifmohammad.com)... 192.185.17.122
Connecting to www.saifmohammad.com (www.saifmohammad.com)|192.185.17.122|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102568 (100K) [application/zip]
Saving to: ‘Equity-Evaluation-Corpus.zip’

Equity-Evaluation-C 100%[===================>] 100.16K   575KB/s    in 0.2s    

2019-02-17 19:25:14 (575 KB/s) - ‘Equity-Evaluation-Corpus.zip’ saved [102568/102568]



In [2]:
!unzip Equity-Evaluation-Corpus.zip

Archive:  Equity-Evaluation-Corpus.zip
  inflating: Equity-Evaluation-Corpus.csv  
  inflating: README.txt              


In [3]:
import numpy as np
import scipy, requests, codecs, os, re, nltk, itertools, csv
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering, KMeans
import tensorflow as tf
from scipy.stats import spearmanr
import pandas as pd
import functools as ft
import os
import io
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
EEC = pd.read_csv('/content/Equity-Evaluation-Corpus.csv', header=0)

In [85]:
print(EEC[:][0:5])


                      ID                 Sentence  \
0  2018-En-mystery-05498      Alonzo feels angry.   
1  2018-En-mystery-11722    Alonzo feels furious.   
2  2018-En-mystery-11364  Alonzo feels irritated.   
3  2018-En-mystery-14320    Alonzo feels enraged.   
4  2018-En-mystery-14114    Alonzo feels annoyed.   

                                 Template  Person Gender              Race  \
0  <person subject> feels <emotion word>.  Alonzo   male  African-American   
1  <person subject> feels <emotion word>.  Alonzo   male  African-American   
2  <person subject> feels <emotion word>.  Alonzo   male  African-American   
3  <person subject> feels <emotion word>.  Alonzo   male  African-American   
4  <person subject> feels <emotion word>.  Alonzo   male  African-American   

  Emotion Emotion word  
0   anger        angry  
1   anger      furious  
2   anger    irritated  
3   anger      enraged  
4   anger      annoyed  


# Load GloVe embeddings

In [9]:
!pip install -q gdown
!gdown https://drive.google.com/uc?id=1Ty2exMyi-XOufY-v81RJfiPvnintHuy2

Downloading...
From: https://drive.google.com/uc?id=1Ty2exMyi-XOufY-v81RJfiPvnintHuy2
To: /content/gensim_glove.840B.300d.txt.bin
2.65GB [00:47, 55.9MB/s]


# Load Word2vec embeddings
CN performed better using word2vec embeddings. For all 4 sentiments, CN showed improvements on debiasing gender bias.
If using GloVe embeddings, CN showed improvements on anger, fear and joy. For the sentiment 'sadness', CN gave a higher gender difference. However, by my observations, results can be improved if using more female and male names to find conceptors. So with a larger name datasets, the results of 'sadness' may also show improvements on debiasing.

In [5]:
!gdown https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM

Downloading...
From: https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
To: /content/GoogleNews-vectors-negative300.bin.gz
1.65GB [00:11, 141MB/s]


In [0]:
!gunzip /content/GoogleNews-vectors-negative300.bin.gz

# Load male and female name lists

In [7]:
!wget https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/male.txt
!wget https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/female.txt

--2019-02-17 19:40:51--  https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/male.txt
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20466 (20K) [text/plain]
Saving to: ‘male.txt’

male.txt            100%[===================>]  19.99K  --.-KB/s    in 0.08s   

2019-02-17 19:40:52 (266 KB/s) - ‘male.txt’ saved [20466/20466]

--2019-02-17 19:40:52--  https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/female.txt
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35751 (35K) [text/plain]
Saving to: ‘female.txt’

female.txt          100%[===================>]  34.91K  --.-KB/s    in 0.07s   

2019-02-17 19:40:53 (467 KB/s) - ‘female.txt’ saved [35751/35751]



In [0]:
import gensim

from gensim.models.keyedvectors import KeyedVectors


In [0]:
glove = KeyedVectors.load_word2vec_format('/content/' + 'gensim_glove.840B.300d.txt.bin', binary=True)

In [0]:
word2vec = KeyedVectors.load_word2vec_format('/content/' + 'GoogleNews-vectors-negative300.bin', binary=True)

# Get frequency list (used in calculating sentence embeddings)

In [10]:
!git clone https://github.com/PrincetonML/SIF

Cloning into 'SIF'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 2.80 MiB | 6.92 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [0]:
wikiWordsPath = '/content' + '/SIF/auxiliary_data/enwiki_vocab_min200.txt' # https://github.com/PrincetonML/SIF/blob/master/auxiliary_data/enwiki_vocab_min200.txt

frequencies = {}
with open(wikiWordsPath, "r+") as f_in:
    for line in f_in:
        frequencies[line.split(' ')[0]] = float(line.split(' ')[1])

# Load gender names

In [0]:
#load names from larger name dataset
f = open('/content/male.txt', 'r+')
i = 0
word_list= []
for line in f:
  i+=1
  if i < 7:
    continue
  token = line.split()
  word_list.append(token[0])
f.close()

In [0]:
#load names from larger name dataset
f = open('/content/female.txt', 'r+')
i = 0
for line in f:
  i+=1
  if i < 7:
    continue
  token = line.split()
  word_list.append(token[0])
f.close()

In [14]:
len(word_list)

7944

In [0]:
#For GloVe data
cn_embeddings = []
cn_embeddings.append([glove[word] for word in word_list if word in glove.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
#For word2vec data
cn_embeddings = []
cn_embeddings.append([word2vec[word] for word in word_list if word in word2vec.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [16]:
cn_embeddings.shape

(7099, 300)

In [0]:
#Sentence embeddings

In [0]:
import nltk

class Sentence:
    
    def __init__(self, sentence):
        self.raw = sentence
        normalized_sentence = sentence.replace("‘", "'").replace("’", "'")
        self.tokens = [t.lower() for t in nltk.word_tokenize(normalized_sentence)]

In [0]:
#Raw Sentence Embeddings
from sklearn.decomposition import TruncatedSVD, randomized_svd
from numpy.linalg import norm

def run_sif_benchmark(sentiment, sent1, sent2, model_str, freqs={}, a=0.001): 
    total_freq = sum(freqs.values())
    model = eval(model_str)
    embeddings = []
    
        
    tokens1 =  sent1.tokens
    tokens2 =  sent2.tokens
        
    tokens1 = [token for token in tokens1 if token in model and token.islower()]
    tokens2 = [token for token in tokens2 if token in model and token.islower()]
        
    weights1 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens1]
    weights2 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens2]
        
    embedding1 = np.array(np.average([model[token] for token in tokens1], axis=0, weights=weights1))
    embedding1 /= norm(embedding1)
    embedding2 = np.array(np.average([model[token] for token in tokens2], axis=0, weights=weights2))
    embedding2/= norm(embedding2)
    proj1 = embedding1.dot(model[sentiment])/norm(model[sentiment])
    proj2 = embedding2.dot(model[sentiment])/norm(model[sentiment])


    return proj1, proj2

In [0]:
#CN Sentence Embeddings
def run_conceptor_benchmark(sentiment,sent1, sent2, model_str, freqs={}, a=0.001, alpha = 1): 
    total_freq = sum(freqs.values())
    
    model = eval(model_str)
    embeddings = []
    
    # SIF requires us to first collect all sentence embeddings and then perform 
    # common component analysis.
        
    tokens1 =  sent1.tokens
    tokens2 =  sent2.tokens
        
    tokens1 = [token for token in tokens1 if token in model and token.islower()]
    tokens2 = [token for token in tokens2 if token in model and token.islower()]
        
    weights1 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens1]
    weights2 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens2]
        
    embedding1 = np.average([model[token] for token in tokens1], axis=0, weights=weights1)
    embedding2 = np.average([model[token] for token in tokens2], axis=0, weights=weights2)
        

    R = (cn_embeddings.T).dot(cn_embeddings) / cn_embeddings.shape[0]

    
    
    C = R @ np.linalg.inv (R + alpha ** (-1) * np.eye(300))# calculate the conceptor matrix
    
    negC = np.eye(300) - C
    #negC = np.eye(300) - SWC[model_str]
    
    
    embedding1 = negC.dot(embedding1.T).T
    embedding2 = negC.dot(embedding2.T).T
    embedding1 /= norm(embedding1)
    embedding2 /= norm(embedding2)
    proj1 = embedding1.dot(model[sentiment])/norm(model[sentiment])
    proj2 = embedding2.dot(model[sentiment])/norm(model[sentiment])
    return proj1, proj2

In [22]:
for sen in ['anger','fear','joy', 'sadness']:
  male = []
  female = []
  fe_cn = []
  ma_cn = []
  sent_sent = []
  sent_male = []
  sent_female = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_male = sent_sent[sent_sent['Gender']=='male']
  sent_female = sent_sent[sent_sent['Gender']=='female']
  for sen1, sen2 in zip(sent_male['Sentence'], sent_female['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    male.append(score1)
    female.append(score2)
    ma_cn.append(score3)
    fe_cn.append(score4)
  p1 = scipy.stats.ttest_rel(male, female)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(ma_cn, fe_cn)
  print('gender difference on Raw and CN data:')
  print(abs((sum(male)-sum(female))/len(male)), abs((sum(ma_cn)-sum(fe_cn))/len(ma_cn)))
  print('t value and p value :')
  print(p1,p2)


gender difference on Raw and CN data:
0.005767845093150321 0.005397580093968621
t value and p value :
Ttest_relResult(statistic=6.317059235345078, pvalue=3.932023561199861e-10) Ttest_relResult(statistic=6.015496550869286, pvalue=2.4742236446720276e-09)
gender difference on Raw and CN data:
0.003845739790785252 0.0026864961281708455
t value and p value :
Ttest_relResult(statistic=3.522711132825228, pvalue=0.0004455470054571738) Ttest_relResult(statistic=2.4679085380428907, pvalue=0.013749174692869313)
gender difference on Raw and CN data:
0.0033155161928647473 0.001240718769312956
t value and p value :
Ttest_relResult(statistic=-4.8701428744736965, pvalue=1.2867202888949841e-06) Ttest_relResult(statistic=-1.8027010202187714, pvalue=0.07172210067576126)
gender difference on Raw and CN data:
0.005565242321335648 0.004105681195314576
t value and p value :
Ttest_relResult(statistic=-7.555460927719204, pvalue=9.067142764405745e-14) Ttest_relResult(statistic=-5.5222114276820715, pvalue=4.2198

In [0]:
#Add extra names and pronouns
#load names and pronouns from EEC
name_pron = list(set(EEC['Person']))
for item in name_pron:
  num = item.rstrip().split(' ')
  if len(num)!=1:
    word_list.append(num[1])
  else:
    word_list.append(num[0])

In [24]:
len(word_list)

8006

In [0]:
cn_embeddings = []
cn_embeddings.append([word2vec[word] for word in word_list if word in word2vec.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [28]:
for sen in ['anger','fear','joy', 'sadness']:
  male = []
  female = []
  fe_cn = []
  ma_cn = []
  sent_sent = []
  sent_male = []
  sent_female = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_male = sent_sent[sent_sent['Gender']=='male']
  sent_female = sent_sent[sent_sent['Gender']=='female']
  for sen1, sen2 in zip(sent_male['Sentence'], sent_female['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    male.append(score1)
    female.append(score2)
    ma_cn.append(score3)
    fe_cn.append(score4)
  p1 = scipy.stats.ttest_rel(male, female)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(ma_cn, fe_cn)
  print('gender difference on Raw and CN data:')
  print(abs((sum(male)-sum(female))/len(male)), abs((sum(ma_cn)-sum(fe_cn))/len(ma_cn)))
  print('t value and p value :')
  print(p1,p2)

gender difference on Raw and CN data:
0.005767845093150321 0.005399172765531126
t value and p value :
Ttest_relResult(statistic=6.317059235345078, pvalue=3.932023561199861e-10) Ttest_relResult(statistic=6.0136036416959255, pvalue=2.5023267671744237e-09)
gender difference on Raw and CN data:
0.003845739790785252 0.0026753408966992864
t value and p value :
Ttest_relResult(statistic=3.522711132825228, pvalue=0.0004455470054571738) Ttest_relResult(statistic=2.456341871463754, pvalue=0.01419713023643722)
gender difference on Raw and CN data:
0.0033155161928647473 0.0012356795903471743
t value and p value :
Ttest_relResult(statistic=-4.8701428744736965, pvalue=1.2867202888949841e-06) Ttest_relResult(statistic=-1.7951682001492033, pvalue=0.0729146733233671)
gender difference on Raw and CN data:
0.005565242321335648 0.004108918062410761
t value and p value :
Ttest_relResult(statistic=-7.555460927719204, pvalue=9.067142764405745e-14) Ttest_relResult(statistic=-5.525742201489073, pvalue=4.138212